## See [Keras: Sequence to sequence](https://keras.io/examples/lstm_seq2seq/)

In [1]:
%env KERAS_BACKEND=tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training.
epochs = 300  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 500  # Number of samples to train on. # total 21007 samples
# Path to the data txt file on disk.
data_path = 'json_h5/RNN_translate.txt'

In [3]:
# Vectorize the data.
input_texts = [] # list of input sentences
target_texts = [] # list of output sentences
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 500
Number of unique input tokens: 58
Number of unique output tokens: 538
Max sequence length for inputs: 13
Max sequence length for outputs: 11


In [5]:
# char to int
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#---------------------seems like one hot encoding---------------------------
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
#---------------------seems like one hot encoding---------------------------

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, 
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle=True, 
          validation_split=0.2
         )
# Save model
# model.save('json_h5/RNN_translate.h5')
model_json = model.to_json()
open('json_h5/RNN_translate.json','w').write(model_json)
model.save_weights('json_h5/RNN_translate.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 58)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 538)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 322560      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  814080      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

400/400 [==============================] - 1s 2ms/step - loss: 1.3644 - acc: 0.2348 - val_loss: 2.9604 - val_acc: 0.2118
Epoch 46/300
400/400 [==============================] - 1s 2ms/step - loss: 1.3700 - acc: 0.2395 - val_loss: 2.9888 - val_acc: 0.2218
Epoch 47/300
400/400 [==============================] - 1s 2ms/step - loss: 1.3177 - acc: 0.2459 - val_loss: 3.0188 - val_acc: 0.2173
Epoch 48/300
400/400 [==============================] - 1s 2ms/step - loss: 1.3090 - acc: 0.2466 - val_loss: 2.9956 - val_acc: 0.2236
Epoch 49/300
400/400 [==============================] - 1s 2ms/step - loss: 1.2846 - acc: 0.2489 - val_loss: 3.0996 - val_acc: 0.2155
Epoch 50/300
400/400 [==============================] - 1s 2ms/step - loss: 1.2603 - acc: 0.2516 - val_loss: 2.9979 - val_acc: 0.2209
Epoch 51/300
400/400 [==============================] - 1s 2ms/step - loss: 1.2672 - acc: 0.2511 - val_loss: 2.9273 - val_acc: 0.2245
Epoch 52/300
400/400 [==============================] - 1s 2ms/step - loss:

400/400 [==============================] - 1s 2ms/step - loss: 0.3376 - acc: 0.4611 - val_loss: 3.0293 - val_acc: 0.2418
Epoch 107/300
400/400 [==============================] - 1s 2ms/step - loss: 0.3297 - acc: 0.4616 - val_loss: 3.1118 - val_acc: 0.2427
Epoch 108/300
400/400 [==============================] - 1s 2ms/step - loss: 0.3357 - acc: 0.4580 - val_loss: 3.0492 - val_acc: 0.2327
Epoch 109/300
400/400 [==============================] - 1s 2ms/step - loss: 0.2947 - acc: 0.4714 - val_loss: 3.0541 - val_acc: 0.2445
Epoch 110/300
400/400 [==============================] - 1s 2ms/step - loss: 0.3029 - acc: 0.4645 - val_loss: 3.1286 - val_acc: 0.2373
Epoch 111/300
400/400 [==============================] - 1s 2ms/step - loss: 0.2958 - acc: 0.4682 - val_loss: 3.0615 - val_acc: 0.2382
Epoch 112/300
400/400 [==============================] - 1s 2ms/step - loss: 0.2776 - acc: 0.4725 - val_loss: 3.0977 - val_acc: 0.2418
Epoch 113/300
400/400 [==============================] - 1s 2ms/step 

Epoch 167/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0832 - acc: 0.5014 - val_loss: 3.3072 - val_acc: 0.2455
Epoch 168/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0895 - acc: 0.4991 - val_loss: 3.2514 - val_acc: 0.2491
Epoch 169/300
400/400 [==============================] - 1s 2ms/step - loss: 0.1027 - acc: 0.4952 - val_loss: 3.2361 - val_acc: 0.2436
Epoch 170/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0871 - acc: 0.5018 - val_loss: 3.2313 - val_acc: 0.2527
Epoch 171/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0744 - acc: 0.5036 - val_loss: 3.2563 - val_acc: 0.2418
Epoch 172/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0786 - acc: 0.5027 - val_loss: 3.2203 - val_acc: 0.2473
Epoch 173/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0841 - acc: 0.5007 - val_loss: 3.2870 - val_acc: 0.2427
Epoch 174/300
400/400 [==============================] 

400/400 [==============================] - 1s 2ms/step - loss: 0.0391 - acc: 0.5145 - val_loss: 3.3975 - val_acc: 0.2455
Epoch 228/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0424 - acc: 0.5141 - val_loss: 3.4371 - val_acc: 0.2445
Epoch 229/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0407 - acc: 0.5130 - val_loss: 3.4479 - val_acc: 0.2318
Epoch 230/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0585 - acc: 0.5080 - val_loss: 3.3811 - val_acc: 0.2455
Epoch 231/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0334 - acc: 0.5159 - val_loss: 3.4147 - val_acc: 0.2418
Epoch 232/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0349 - acc: 0.5145 - val_loss: 3.4066 - val_acc: 0.2482
Epoch 233/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0321 - acc: 0.5157 - val_loss: 3.4201 - val_acc: 0.2482
Epoch 234/300
400/400 [==============================] - 1s 2ms/step 

Epoch 288/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0294 - acc: 0.5166 - val_loss: 3.5142 - val_acc: 0.2382
Epoch 289/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0309 - acc: 0.5161 - val_loss: 3.4970 - val_acc: 0.2455
Epoch 290/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0179 - acc: 0.5198 - val_loss: 3.4805 - val_acc: 0.2473
Epoch 291/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0190 - acc: 0.5189 - val_loss: 3.4689 - val_acc: 0.2464
Epoch 292/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0187 - acc: 0.5191 - val_loss: 3.5130 - val_acc: 0.2436
Epoch 293/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0196 - acc: 0.5198 - val_loss: 3.5483 - val_acc: 0.2455
Epoch 294/300
400/400 [==============================] - 1s 2ms/step - loss: 0.0207 - acc: 0.5191 - val_loss: 3.5394 - val_acc: 0.2445
Epoch 295/300
400/400 [==============================] 

c:\users\奎佑\appdata\local\programs\python\python36\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [7]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Run.
Decoded sentence: 你用跑的。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I try.
Decoded sentence: 让我来。

-
Input sentence: I won!
Decoded sentence: 我赢了。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 乾杯!

-
Input sentence: Got it?
Decoded sentence: 你懂了吗？

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 跳进来。

-
Input sentence: I lost.
Decoded sentence: 我迷失了。

-
Input sentence: I quit.
Decoded sentence: 我退出。

-
Input sentence: I'm OK.
Decoded sentence: 我沒事。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 不可能！

-
Input sentence: No way!
Decoded sentence: 不可能！

-
Input sentence: Really?
Decoded sentence: 你确定？

-
Input sentence: Try it.
Decoded sentence: 试试吧。

-
Input sentence: We